In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
from sklearn.model_selection import train_test_split as tts

In [2]:
t.manual_seed(4)
np.random.seed(4)

def create_dataset_1(n):
    X,Y = [],[]
    for _ in range(2000):
        r = np.random.randint(2,n)
        x = np.random.uniform(0,1,size=(2,r))
        x2 = np.zeros(r)
        x2[0:2] = 1
        x[1,:] = np.random.permutation(x2)
        X.append(x)
        Y.append(np.dot(x[0,:],x[1,:]))
    return X,Y


def create_dataset_2(n):
    X,Y = [],[]
    for _ in range(2000):
        r = np.random.randint(2,n)
        x = np.random.uniform(0,1,size=(2,r))
        x2 = np.zeros(r)
        x2[0] = 1
        x2[1] = -1
        x[1,:] = np.random.permutation(x2)
        X.append(x)
        Y.append(np.dot(x[0,:],x[1,:]))
        Y.append(np.sum(2*x[0,:]+5*x[1,:]))
    return X,Y



In [3]:
class gru(nn.Module):
    def __init__(self, ins, hs, os):
        super(gru, self).__init__()
        self.hs = hs
        self.wz = nn.Linear(ins + hs,hs)
        self.wr = nn.Linear(ins + hs,hs)
        self.wh = nn.Linear(ins + hs,hs)
        self.hy = nn.Linear(hs,os)

    def Sigmoid(self,x):
        return 1/(1+t.exp(-1*x))

    def init_hidden(self):
        self.h = t.zeros(1,self.hs)

    def forward_one_pair(self,x):
        ip = t.cat((x,self.h),1)
        z = self.Sigmoid(self.wz(ip))
        r = self.Sigmoid(self.wr(ip))
        ip1 = t.cat((x,r*self.h),1)
        h1 = t.tanh(self.wh(ip1))
        self.h = (1-z)*self.h + z*h1
        output = self.hy(self.h)
        return output
    
    def forward(self,x):
        self.init_hidden()
        for a in range(x.shape[1]):
            output = self.forward_one_pair(x.T[a].reshape(1,2))
        return output

In [4]:
def train(net,optimizer,loss_fn,xtrain,ytrain,epochs):
    res_array = []
    for i in range(epochs):
        loss1 = 0
        net.train()
        for x,y in zip(xtrain,ytrain):
            optimizer.zero_grad()
            X = t.tensor(x).to(t.float32)
            Y = t.tensor(y).to(t.float32)
            output = net(X).squeeze(0)
            loss = loss_fn(output,Y)
            loss.backward()
            optimizer.step()
            loss1 += loss.item()
        res_array.append(loss1)
        print(f"Training loss at iteration {i} is {loss1/len(ytrain)}")
    return net,res_array

In [5]:
def accuracy(net,xtest,ytest):
    ypred = []
    for a in xtest:
        a = t.tensor(a).to(t.float32)
        u = net(a).squeeze(0).item()
        ypred.append(u)
    ypred = np.array(ypred)
    return round(((abs(ypred-ytest)<=0.017).sum() / len(ypred))*100,4)

Defining Models


In [6]:
X,Y = create_dataset_1(9)
X1,Y1 = create_dataset_2(9)
xtrain,xtest,ytrain,ytest=tts(X,Y,random_state=42,test_size=0.15,shuffle=True)

In [7]:
t.manual_seed(4)
np.random.seed(4)

epochs = 15
net1 = gru(2,6,1)
optimizer1 = t.optim.Adam(net1.parameters(),lr=0.005)
loss_fn1 = t.nn.MSELoss()
net1,res_array1 = train(net1,optimizer1,loss_fn1,xtrain,ytrain,epochs)

/home/nk/.conda/envs/code/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Training loss at iteration 0 is 0.08346007551101546
Training loss at iteration 1 is 0.003949735951300922
Training loss at iteration 2 is 0.0007280590762217124
Training loss at iteration 3 is 0.00047562723286135087
Training loss at iteration 4 is 0.0005031030432972685
Training loss at iteration 5 is 0.0005033874159442866
Training loss at iteration 6 is 0.0004829774015939638
Training loss at iteration 7 is 0.0004591281831692071
Training loss at iteration 8 is 0.0004354198530231024
Training loss at iteration 9 is 0.0004128205137806371
Training loss at iteration 10 is 0.00039204789733066504
Training loss at iteration 11 is 0.0003738377144406153
Training loss at iteration 12 is 0.000358460727495728
Training loss at iteration 13 is 0.0003457319035981269
Training loss at iteration 14 is 0.00033528869840021217


In [8]:
print(f"The Accuracy on testing data for GRU is {accuracy(net1,xtest,ytest)}")

The Accuracy on testing data for GRU is 91.0


In [ ]:
xtrain1,xtest1,ytrain1,ytest1=tts(X1,Y1,random_state=42,test_size=0.15,shuffle=True)